# Homework 2

## FINM 37400 - 2025

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

***

# 1 HBS Case: Fixed-Income Arbitrage in a Financial Crisis (A): US Treasuries in November 2008

## Data
* Use the data file `treasury_ts_2015-08-15.xlsx`.
* Examine the treasure issues with `kytreasno` of `204046` and `204047`. These are the bond and note (respectively) which mature on 2015-08-15.
* Look at the data on 2008-11-04.

In [8]:
# PACKAGES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os 
from pyprojroot import here
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

# SETTINGS
pd.set_option('display.float_format', '{:.4f}'.format) # Limiting floats output to 4 decimal points

# Set working directory
os.chdir("/Users/charleston/Downloads/git_repositories/finm-fixedincome-2025")

# DATE
quote_date = '2015-08-15'

# DATA
treasury_ts = pd.read_excel(here(f"./data/treasury_ts_{quote_date}.xlsx"), sheet_name="database")
treasury_ts.query('kytreasno in [204046, 204047]', inplace=True)
treasury_ts.set_index('caldt', inplace=True)

info = pd.read_excel(here(f"./data/treasury_ts_{quote_date}.xlsx"), sheet_name="info", skiprows=1)
info = info[['kytreasno', 204046, 204047]]


## 1.1 The situation

Make a chart comparing the issues in the following features, (as of Nov 4, 2008.)
* coupon rate
* bid
* ask
* accrued interest
* dirty price
* duration (quoted in years, not days, assuming 365.25 days per year.)
* modified duration
* YTM

In [49]:
# Extract relevant columns for the specified date
data_nov_4_2008 = treasury_ts.loc['2008-11-04']

# Create a new DataFrame for the chart
chart = pd.DataFrame({
    'Coupon Rate': info[info['kytreasno'] == 'tcouprt'].iloc[0, 1:].values[::-1], # In reverse order to match the chart
    'Bid': data_nov_4_2008['tdbid'].values,
    'Ask': data_nov_4_2008['tdask'].values,
    'Accrued Interest': data_nov_4_2008['tdaccint'].values,
    'Dirty Price': (data_nov_4_2008['tdbid'].values + data_nov_4_2008['tdask'].values) / 2 + data_nov_4_2008['tdaccint'].values,
    'Duration': (data_nov_4_2008['tdduratn']/365.25).values,
    'Modified Duration': ((data_nov_4_2008['tdduratn']/365.25)/(1+data_nov_4_2008['tdyld']*365.25/2)).values,
    'YTM': (data_nov_4_2008['tdyld']*365.25).values
}, index=['Notes', 'Bonds'])

# Display the chart DataFrame
chart.T

,Notes,Bonds
Coupon Rate,4.2500,10.6250
Bid,105.9531,141.8594
Ask,105.9844,141.8906
Accrued Interest,0.9355,2.3387
Dirty Price,106.9042,144.2137
Duration,5.9357,5.2301
Modified Duration,5.8412,5.1383
YTM,0.0324,0.0358


## 1.2 Hedge Ratio

Suppose you are building a trade to go long $n_i$ bonds (`204046`) and short $n_j$ notes (`204047`).

We can find an equation for $n_j$ in terms of $n_i$ such that the total holdings will have duration equal to zero. (Having zero duration also means have zero dollar duration, if helpful.)

Notation:
* $n_i$: number of bonds purchased (or sold)
* $D_i$: duration of bond $i$
* $D_{\$,i}$: dollar duration of bond $i$, equal to $p_iD_i$

If we want the total duration of our holdings to be zero, then we need to size the trade such that $n_i$ and $n_j$ satisfy,

$$0 = n_iD_{\$,i} + n_jD_{\$,j}$$

$$n_j = -n_i\frac{D_{\$,i}}{D_{\$,j}}$$

Suppose you will use \\$1mm of capital, leveraged 50x to buy \\$50mm of the bonds (`204046`).

Use the ratio above to short a number of notes (`204047`) to keep zero duration.

Report the number of bonds and notes of your position, along with the total dollars in the short position.

In [45]:
hedge_ratio = pd.DataFrame(columns=['Equity', 'Assets', 'Contracts'], index=['Bonds', 'Notes'])

hedge_ratio.loc['Bonds', 'Equity'] = 1_000_000
hedge_ratio.loc['Bonds', 'Assets'] = 50_000_000
hedge_ratio.loc['Bonds', 'Contracts'] = hedge_ratio.loc['Bonds', 'Assets'] / chart.loc[204046, 'Dirty Price']

hedge_ratio.loc['Notes', 'Contracts'] = - hedge_ratio.loc['Bonds', 'Contracts'] * ((chart.loc[204046, 'Duration']*chart.loc[204046, 'Dirty Price'])/ (chart.loc[204047, 'Duration']*chart.loc[204047, 'Dirty Price']))
hedge_ratio.loc['Notes', 'Assets'] = hedge_ratio.loc['Notes', 'Contracts'] * chart.loc[204047, 'Dirty Price']
hedge_ratio.loc['Notes', 'Equity'] = hedge_ratio.loc['Notes', 'Assets']/50

# Format the DataFrame
hedge_ratio_formatted = hedge_ratio.copy()
hedge_ratio_formatted['Equity'] = hedge_ratio_formatted['Equity'].apply(lambda x: f"${x:,.2f}")
hedge_ratio_formatted['Assets'] = hedge_ratio_formatted['Assets'].apply(lambda x: f"${x:,.2f}")
hedge_ratio_formatted['Contracts'] = hedge_ratio_formatted['Contracts'].apply(lambda x: f"{x:,.2f}")

# Display the formatted DataFrame
hedge_ratio_formatted

,Equity,Assets,Contracts
Bonds,"$1,000,000.00","$50,000,000.00","346,707.81"
Notes,"$-881,131.58","$-44,056,578.93","-412,112.66"


## 1.3 Profit Opportunity

Using the concept of **modified duration**, how much profit or loss (PnL) would you expect to make for every basis point of convergence in the spread? Specifically, assume the convergence is symmetric: the bond's (`204046`) ytm goes down 0.5bp and the note (`204047`) ytm goes up 0.5bp.

Describe the PnL you would expect to achieve on your position should this happen. Specify the PnL of the long position, the short position, and the net total.

Suppose the spread in YTM between the two securities disappears, due to a symmetric move of roughly ~17bps in each security's YTM. What is the PnL? (This is just a linearly scaling of your prior answer for a 1bp convergence.) 

In [125]:
# Modified duration is mathematically the elasticity of price with respect to the YTM. 
# The pnl that we would expect will be given by the modified duration, times the change in ytm, times the initial price.

profit_opportunity = pd.DataFrame(columns=['YTM Change', 'Modified Duration', 'Initial Price', 'Contracts', 'pnl'], index=['Bonds', 'Notes'])

profit_opportunity['YTM Change'] = [-0.00005, +0.00005]
profit_opportunity['Modified Duration'] = chart['Modified Duration']
profit_opportunity['Initial Price'] = chart['Dirty Price']
profit_opportunity['Contracts'] = hedge_ratio['Contracts'].astype(float)
profit_opportunity['pnl'] = -profit_opportunity['Modified Duration'] * profit_opportunity['YTM Change'] * profit_opportunity['Initial Price'] * profit_opportunity['Contracts']

total_profit = profit_opportunity['pnl'].sum()

# Display the DataFrame with formatted columns
profit_opportunity.style.format({
    'YTM Change': '{:.4%}',
    'Modified Duration': '{:.4f}',
    'Initial Price': '${:,.2f}',
    'Contracts': '{:,.2f}',
    'pnl': '${:,.2f}'
})

,YTM Change,Modified Duration,Initial Price,Contracts,pnl
Bonds,-0.0050%,5.1383,$144.21,"346,707.81","$12,845.71"
Notes,0.0050%,5.8412,$106.90,"-412,112.66","$12,867.14"


In [126]:
# Get the mid point of the YTM, and then get the change needed in YTM to converge to the mid point
converges = pd.DataFrame(columns=['YTM Change', 'Modified Duration', 'Initial Price', 'Contracts', 'pnl'], index=['Bonds', 'Notes'])

converges['YTM Change'] = ((chart['YTM'] - chart['YTM'].mean())).values
converges['Modified Duration'] = chart['Modified Duration']
converges['Initial Price'] = chart['Dirty Price']
converges['Contracts'] = hedge_ratio['Contracts'].astype(float)
converges['pnl'] = -converges['Modified Duration'] * converges['YTM Change'] * converges['Initial Price'] * converges['Contracts']

total_profit_converges = converges['pnl'].sum()

# Display the DataFrame with formatted columns
converges.style.format({
    'YTM Change': '{:.4%}',
    'Modified Duration': '{:.4f}',
    'Initial Price': '${:,.2f}',
    'Contracts': '{:,.2f}',
    'pnl': '${:,.2f}'
})

,YTM Change,Modified Duration,Initial Price,Contracts,pnl
Bonds,-0.1695%,5.1383,$144.21,"346,707.81","$435,509.58"
Notes,0.1695%,5.8412,$106.90,"-412,112.66","$436,236.08"


In [127]:
print(f"The net total profit and loss from the narrowing of spread is ${total_profit:,.2f}")
print(f"The net total profit and loss from the convergence is ${total_profit_converges:,.2f}")

The net total profit and loss from the narrowing of spread is $25,712.85
The net total profit and loss from the convergence is $871,745.67


## 1.4 Result in 2008

Calculate the profit (or loss) on the position on the following two dates:
* 2008-11-25
* 2008-12-16

To calculate the pnl on each date, simply use the prices of the securities on those dates along with your position sizes, ($n_i, n_j$). No coupon is being paid in November or December, so all you need is the "dirty" price on these two dates.

Does the pnl make sense (approximately) given your results in 1.3 with regard to the sensitivity of pnl to moves in the YTM spread?


## 1.5 Examining the Trade through June 2009

Calculate the pnl of the trade for the following dates:
* 2009-01-27
* 2009-03-24
* 2009-06-16

Did the trade do well or poorly in the first six months of 2009?

Calculate the YTM spreads on these dates. Does the YTM spread correspond to pnl roughly as we would expect based on the calculation in 1.3?

***

# 2 Hedging Duration

Use data from `../data/treasury_ts_duration_2024-10-31.xlsx`.

The file contains time-series information on two treasuries. Observe the info of the securities with the following code:


In [1]:
import pandas as pd

In [2]:
QUOTE_DATE = '2024-10-31'
filepath = f'../data/treasury_ts_duration_{QUOTE_DATE}.xlsx'

data = pd.read_excel(filepath,sheet_name='database')
data_info =  data.drop_duplicates(subset='KYTREASNO', keep='first').set_index('KYTREASNO')
data_info[['type','issue date','maturity date','cpn rate']]

,type,issue date,maturity date,cpn rate
KYTREASNO,,,,
207391,note,2019-08-15,2029-08-15,1.625
207392,bond,2019-08-15,2049-08-15,2.250


You will largely focus on the sheets which give the timeseries of prices and durations for each of the two securities, as shown in the following code.

In [3]:
SHEET_PRICE = 'price'
SHEET_DURATION = 'duration'
INDEX_NAME = 'quote date'

price = pd.read_excel(filepath,sheet_name=SHEET_PRICE).set_index(INDEX_NAME)
duration = pd.read_excel(filepath,sheet_name=SHEET_DURATION).set_index(INDEX_NAME)

display(price)
display(duration)

,207391,207392
quote date,,
2019-08-09,98.882812,99.789062
2019-08-12,99.796875,102.554688
2019-08-13,99.281250,101.867188
2019-08-14,100.406250,105.179688
2019-08-15,100.882812,106.234375
...,...,...
2024-11-22,88.740234,63.722656
2024-11-25,89.287109,65.378906
2024-11-26,89.214844,65.175781


,207391,207392
quote date,,
2019-08-09,1.237593e+06,2.930964e+06
2019-08-12,1.237056e+06,2.946737e+06
2019-08-13,1.236370e+06,2.942182e+06
2019-08-14,1.236694e+06,2.961398e+06
2019-08-15,1.236616e+06,2.967042e+06
...,...,...
2024-11-22,6.047676e+05,2.292204e+06
2024-11-25,6.037734e+05,2.306427e+06
2024-11-26,6.033936e+05,2.304189e+06


### 2.1.

Suppose you have a portfolio of `10,000` USD long in security `207391` on the first day of the sample.

If you want to manage interest rate exposure using duration, how large of a short position should you hold in `207392`?

### 2.2.

Step through the time-series, doing the following:

* Starting at the end of the first day, set the hedged position according to the relative given durations.
* Use the second day's price data to evaluate the net profit or loss of the hedged position.
* Reset the the hedged position using the end-of-second-day durations. Again fix the long position of security `207391` to be `10,000`.
* Repeat throughout the timeseries.

Calculate the daily profit and loss (PnL) for the
* dynamically hedged position constructed above.
* long-only position, (still at `10,000` throughout.)

(You might check to verify that the net duration is zero at all dates.)

Report...
* the cumulative PnL of both strategies via a plot.
* the (daily) mean, standard deviation, min, and max of the PnL in a table.

### 2.3.

Give two reasons that the daily PnL is not always zero for the hedged position given that we have perfectly hedged the duration.

### 2.4.
The PnL above doesn't account for the coupons.

Calculate a dataframe indexed by dates with columns for the two treasuries with values of coupon payments. 
* Recall that the stated coupon rate is semiannual, so at any give coupon date, it pays half the stated rate.
* Figure out the coupon dates by using the `data` tab and looking for dates where `acc int` goes down. Recall that accrued interest measures the portion of the coupon period that has passed. So when this resets, it is because the coupon has been paid.

Report the first 5 dates that a coupon is paid (by either bond).

### 2.5.
Account for the coupons in the PnL calculations of `2.2`. Report the updated PnL in a plot and a table, similar to the reporting in `2.2`.

***